# Interactive graph visualisation via Bokeh
https://docs.bokeh.org/en/latest/index.html

In [ ]:
# Load the python package
import os
from dynetan.toolkit import *
from dynetan.viz import *
from dynetan.proctraj import *
from dynetan.gencor import *
from dynetan.contact import *
from dynetan.datastorage import *

from MDAnalysis.analysis import distances as MDAdistances
#from numpy.linalg import norm
#from itertools import islice
from itertools import combinations
from scipy import stats
from collections import OrderedDict

import networkx as nx
import numpy as np
import scipy as sp

In [ ]:
# For visualization

from bokeh.io import output_file, output_notebook, push_notebook, show
from bokeh import models as bokehModels
from bokeh import transform as bokehTransform
from bokeh import layouts as bokehLayouts
from bokeh import plotting as bokehPlotting
from bokeh import palettes as bokehPalettes
from bokeh import events as bokehEvents
# For pre-calculating CArtesian distances based on 2D embedding
from sklearn.manifold import MDS

In [ ]:
mapResidueNames={'ALA':'A','CYS':'C','ASP':'D','GLU':'E','PHE':'F',
                 'GLY':'G','HIS':'H','HSD':'H','HSE':'H','ILE':'I','LYS':'K','LEU':'L',
                 'MET':'M','ASN':'N','PRO':'P','GLN':'Q','ARG':'R',
                 'SER':'S','THR':'T','VAL':'V','TRP':'W','TYR':'Y',
                 'MG':'Mg','ATP':'Atp','POPC':'Popc','SOL':'h2o'}

def name_node(dnad, node):
    #i=dnad.nodesAtmSel[node].index
    resname=dnad.nodesAtmSel[node].resname ; resid=dnad.nodesAtmSel[node].resid
    return "%s%s" % (mapResidueNames[resname], resid)

def clarify_duplicate_nodes(dictNames, dictSuffix):
    """
    From two dicts with the same keys, add the respective suffix to all keys in the former that possess duplicate values.
    """
    from itertools import chain
    dictRev = {}
    for k, v in dictNames.items():
        dictRev.setdefault(v, set()).add(k)
        setDuplicateKeys = set(chain.from_iterable( v for k, v in dictRev.items() if len(v) > 1))
    for k in setDuplicateKeys:
        dictNames[k] = dictNames[k]+"_"+dictSuffix[k]
    return dictNames  

In [ ]:
def encode_neighbour_information(G):
    # = = Do over each node. First enccode javascript ID, assuming that order is preserved.
    for i, x in enumerate(G.nodes()):
        G.nodes[x]['jsID']=i
    for i, x in enumerate(G.edges()):
        G.edges[x]['jsID']=i
        
    for x in G.nodes():
        G.nodes[x]['jsNeighbours'] = [ G.nodes[y]['jsID'] for y in G.neighbors(x) ]
#        G.nodes[x]['jsEdges'] = [ G.edges[y]['jsID'] for y in G.edges(x) ]

## Files and system definitions

### Load the relevant allele data into a DNAD object

In [ ]:
%cd ..
# Define mutant file IO locations. wt, P67L, E56K, R75Q, S945L, dF508
allele="wt" ; temperature="310K"
dataDir = "./results/%s/%s/" % (allele, temperature)
#Path where results will be written (you may want plots and data files in a new location)
workDir = "./results/%s/%s/analysis" % (allele, temperature)
fileNameRoot = "1to6"
fullPathRoot = os.path.join(dataDir, fileNameRoot)

In [ ]:
bExportHTML = True
outputFileName = "./results/networkView_%s_%s.html" % (allele, temperature)

In [ ]:
dnad = DNAdata()

In [ ]:
dnad.loadFromFile(fullPathRoot)

In [ ]:
dcdVizFile = fullPathRoot + "_reducedTraj.dcd"
pdbVizFile = fullPathRoot + "_reducedTraj.pdb"
mdU = mda.Universe(pdbVizFile,dcdVizFile)
dnad.nodesAtmSel = mdU.atoms[ dnad.nodesIxArray ]

### Load Wildtype data

In [ ]:
# Define mutant file IO locations. wt, P67L, E56K, R75Q, S945L, dF508
if allele != "wt":
    dataWTDir = "./results/wt/%s/" % (temperature)
    fileNameRoot = "1to3"
    fullPathWTRoot = os.path.join(dataWTDir, fileNameRoot)
    dnadWT = DNAdata()
    dnadWT.loadFromFile(fullPathWTRoot)
    dcdVizFile = fullPathWTRoot + "_reducedTraj.dcd"
    pdbVizFile = fullPathWTRoot + "_reducedTraj.pdb"
    mdUWT = mda.Universe(pdbVizFile,dcdVizFile)
    dnadWT.nodesAtmSel = mdUWT.atoms[ dnadWT.nodesIxArray ]
else:
    dnadWT = dnad
    mdUWT = mdU

### Encode Additional Data into nxGraph (most of which should now be done in Step 1 and saved into the graph by default).
1. Clustering Coefficient
2. Node and Edge Betweenness Centrality
3. Node names and SegIDs
4. Edge IDs for each node.

In [ ]:
# Explanation of eigenvector centrality: http://olizardo.bol.ucla.edu/classes/soc-111/textbook/_book/6-6-sec-eigencent.html
listNodeProperties   = ['segid','communityID','eigenvector','btws','bwcc','cwcc']
listNodePropertyNames= ['Seg ID', 'Community ID','Eigenvector centrality','Node betweenness',
                        'Edge-betweenness weighted clustering coefficient',
                        'GCC weighted clustering coefficient']
listNodePropertiesPlot=['eigenvector','btws','bwcc']
listNodePropertiesPlotNames=['Eigenvector centrality','Node betweenness',
                        'Edge-betweenness weighted clustering coefficient']

print("= = Check if graph has additional information not calculated in the default tutorial content.")
for w in range(dnad.numWinds):
    G = dnad.nxGraphs[w]
    testNode=0 ; testEdge=(0,1)
    # Note: btws is the same as the results dervied from nx.edge_betweenness_centrality()
    if not 'btws' in G.edges[testEdge].keys():
        print("   ...encoding edge betweenness into the graph.")
        for u,v in dnad.btws[w]:
            G.edges[u,v]['btws']=dnad.btws[w][u,v]

    # Note: edge-betweeness weighted clustering coefficient
    if not 'bwcc' in G.nodes[testNode].keys():
        print("   ...analysing edge-betweenness weighted clustering coefficient and encoding into graph.")
        nx.set_node_attributes(G, nx.clustering(G, weight='btws'), 'bwcc')
        
    # Note: GCC weighted clustering coefficient
    if not 'cwcc' in G.nodes[testNode].keys():
        print("   ...analysing GCC-weighted clustering coefficient and encoding into graph.")
        nx.set_node_attributes(G, nx.clustering(G, weight='weight'), 'cwcc')
        
    # Note: this is the node betweennes centrality, which might be useful in a different context.
    if not 'btws' in G.nodes[testNode].keys():
        print("   ...analysing node-betweenness and encoding into graph.")
        nx.set_node_attributes(G, nx.betweenness_centrality(G), 'btws')

    if not 'name' in G.nodes[testNode].keys():
        print("   ...naming the nodes of the graph.")
        nodeNames={} ; nodeSegIDs={} ; nodeAtomNames={}
        for x in G.nodes():
            #i=dnad.nodesAtmSel[x].index
            nodeNames[x]  = name_node(dnad, x)
            nodeSegIDs[x] = dnad.nodesAtmSel[x].segid
            nodeAtomNames[x] = dnad.nodesAtmSel[x].name
        nodeNames = clarify_duplicate_nodes( nodeNames, nodeAtomNames )               
        nx.set_node_attributes(G, nodeNames, "name")
        nx.set_node_attributes(G, nodeSegIDs, "segid")
        
    print("   ...finished checking additional graph information for key %s, window %i" % ('wt', w))

In [ ]:
# = = Encode Neighbour information for Javascript referencing in the browser.
for w in range(dnad.numWinds):
    encode_neighbour_information(dnad.nxGraphs[w])

In [ ]:
#pale = bokehPalettes.Viridis256
#pMax = 256
for w in range(dnad.numWinds):
    G=dnad.nxGraphs[w]
    vals = [ dnad.btws[w][x] for x in G.edges() ]
    vMin = np.min(vals) ; vMax=np.max(vals)

    edgeAlphasBtws={} ; edgeAlphasWeight={}
    edgeNames={}
    for x in G.edges():
        #edgeColors[x] = '#000000'
        r = (dnad.btws[w][x]-vMin)/(vMax-vMin)
        edgeAlphasBtws[x] = 0.1+0.9*r
    
    vals = [ G.edges[x]['weight'] for x in G.edges() ]
    vMin = np.min(vals) ; vMax=np.max(vals)
    for x in G.edges():
        r = (G.edges[x]['weight']-vMin)/(vMax-vMin)
        edgeAlphasWeight[x] = 0.1+0.9*r
        edgeNames[x]= "%s-%s" % (G.nodes[x[0]]['name'],G.nodes[x[1]]['name'])

    #nx.set_edge_attributes(G, edgeColors, "edge_color")
    nx.set_edge_attributes(G, edgeAlphasBtws, "edge_alpha")
    nx.set_edge_attributes(G, edgeAlphasBtws, "edge_alpha_btws")
    nx.set_edge_attributes(G, edgeAlphasWeight, "edge_alpha_weight")        
    nx.set_edge_attributes(G, edgeNames, "name")

In [ ]:
#dnad.nodesAtmSel[1300].index
# Undirected case: [G.edges(x) for x in G.nodes()]

In [ ]:
#[ np.mean( [ dnad.nxGraphs[a].nodes[x]['bwcc'] for x in G.nodes() ] ) for a in range(dnad.numWinds) ]
#[ print( dnad.nxGraphs[a].nodes[0] ) for a in range(3) ]
#[ [ len(dnad.nodesComm[a]['commNodes'][x]) for x in dnad.nodesComm[0]['commLabels'] ] for a in range(3) ]

### Load annotations from databases and encode with node data, using node names

In [ ]:
annotationFile1 = "./annotations/resid_annotated_CFTR.csv"
annotationName1 = "CFTR2 DB classification"

In [ ]:
dataAnnotation={}
dataAnnotation[annotationName1] = np.genfromtxt(annotationFile1, delimiter=',', comments='#', dtype=str)

In [ ]:
def annotate_graph(dnad, dataAnnotation):
    G = dnad.nxGraphs[0]
    data = {}
    for x in G.nodes():
        # = = Match resids
        loc = np.where( dataAnnotation[annotationName1][...,0] == G.nodes[x]['name'][1:] )[0]
        if len(loc) == 1 :
            data[x]=dataAnnotation[annotationName1][loc[0],1]    
        elif len(loc) > 1 :
            print( "= = WARNING: A resid search in the annotation database returns multiple results! %s found at %s" % ( name, str(loc)) )
        else:
            data[x]='lack'
    for w in range(dnad.numWinds):
        nx.set_node_attributes(dnad.nxGraphs[w], data, "annotation")

In [ ]:
annotate_graph(dnad, dataAnnotation)
if allele != 'wt':
    annotate_graph(dnadWT, dataAnnotation)

### Compute the consensus graph based on each window

In [ ]:
def compute_consensus_graph(listG, node_properties, edge_properties):
    """
    The node_properties and edge properties are the values to take averages of.
    Assumes nodes are the same between all graphs in the list.
    Assumes that the union of all edges should be used, rather than the intersection of all edges.
    """
    bDoBWCC=False ; bDoCWCC=False  
    nGraphs = len(listG)
    outG = nx.Graph()
    # = = = Create Nodes and Edges = = =
    for x, c in listG[0].nodes.data('name', default=None):
        outG.add_node(x, name=c, communityID=0)
    for w in range(nGraphs):
        for u, v, c in listG[w].edges.data('name', default=None):
            if (u,v) not in outG.edges():
                outG.add_edge(u,v,name=c)
        
    # = = = Evaluate Node Properties = = =        
    for p in node_properties:
        if p == 'btws':
            nx.set_node_attributes(outG, nx.betweenness_centrality(outG), p)
        elif p == 'bwcc':
            bDoBWCC=True
        elif p == 'cwcc':
            bDoCWCC=True
        elif p == 'eigenvector':
            nx.set_node_attributes(outG, nx.eigenvector_centrality(outG), p)
        else:
            dictP={}
            for x in listG[0].nodes:
                dictP[x]=np.mean([ listG[w].nodes[x][p] for w in range(nGraphs) ])
            nx.set_node_attributes(outG, dictP, p)
    
    # = = = Evaluate Edge Properties = = =        
    for p in edge_properties:
        if p == 'btws':
            nx.set_edge_attributes(outG, nx.edge_betweenness_centrality(outG), p)
        else: 
            dictP={}
            for u,v in outG.edges():
                l=[ listG[w].edges[u,v][p] for w in range(nGraphs) if (u,v) in listG[w].edges() ]
                dictP[(u,v)]=np.mean(l)
            nx.set_edge_attributes(outG, dictP, p)

    if bDoBWCC:
        nx.set_node_attributes(outG, nx.clustering(outG, weight='btws'), 'bwcc')
    if bDoCWCC:
        nx.set_node_attributes(outG, nx.clustering(outG, weight='weights'), 'cwcc')

    return outG

In [ ]:
#Gmean = compute_consensus_graph(dnad.nxGraphs, listNodePropertiesPlot, ['btws','dist','bwcc'])
#nx.set_node_attributes(Gmean, dict(dnad.nxGraphs[0].nodes.data('segid')), 'segid')

## Setting the interactive viewer

### Pre-calculate Cartesian coordinates of the graphs in each window
Note: The distsAll key in DNAD is based on -log(Generalised Correlation Coefficient)
    ...we can try embedding using the actual atomic positions in the reduced trajectory.

In [ ]:
def obtain_caretesian_distances(dnad):
    selfDistances = MDAdistances.self_distance_array(dnad.nodesAtmSel.positions)
    sqDistArr = np.zeros((dnad.numNodes, dnad.numNodes))
    triu = np.triu_indices_from(sqDistArr, k=1)
    sqDistArr[triu] = selfDistances
    sqDistArr.T[triu] = selfDistances
    return sqDistArr

In [ ]:
def compute_scaled_cylindrical_coordinates(dnad, offsetTheta=None, bDict=False):
    """
    Remap the cartesian coordiantes of the trajectory into a 2D-space spanning (theta,z),
    with a fixed range (-1,1).
    Optional iffset rotates theta 
    """
    pos = dnad.nodesAtmSel.positions
    pos -= np.mean(pos,axis=0)
    x = np.arctan2(pos[...,1],pos[...,0])
    if offsetTheta is not None:
        x = (x + offsetTheta + np.pi) % (2 * np.pi) - np.pi
    y = np.interp(pos[...,2], (np.min(pos[...,2]), np.max(pos[...,2])), (-1.0, 1.0) )
    xy = np.stack( (x/np.pi,y), axis=-1 )
    if bDict:
        dictPos = {}
        for i, x in enumerate( dnad.nxGraphs[0].nodes() ):
            dictPos[x] = xy[i]
        return dictPos
    else:
        return xy
    
def compute_graph_embedding(distMat, posInit, bMetric=True):
    if bMetric:
        graphEmbedding = MDS(n_components=2, metric=bMetric,
                             max_iter=3000, eps=1e-9,
                             dissimilarity='precomputed', n_init=1)
    else:
        graphEmbedding = MDS(n_components=2, metric=False,
                             max_iter=3000, eps=1e-12,
                             dissimilarity="precomputed", n_init=1)
    graphEmbedding.fit_transform(distMat, init=posInit)
    print(graphEmbedding.embedding_.shape)
    return graphEmbedding.embedding_

def calculate_segID_cogs(G, pos, segIDList):
    nSegs=len(segIDList)
    nodeSegIDs = np.array( [ G.nodes[x]['segid'] for x in G.nodes() ] )
    cogSegs=np.zeros( (nSegs,2) )
    for i, seg in enumerate(segIDList):
        indices = np.where(nodeSegIDs==seg )[0]
        cogSegs[i] = np.mean( np.take(pos,indices,axis=0),axis=0) 
    return cogSegs

def calculate_transform_matrix(cog):
    # First account for mirror imagert
    v1=(cog[3]+cog[1])-(cog[2]+cog[0])
    v2=(cog[0]+cog[1])-(cog[2]+cog[3])
    xx=1.0 if v1[0]>0.0 else -1.0
    yy=1.0 if v2[1]>0.0 else -1.0
    
    M1 = np.array([[xx,0],[0,yy]])
    cog = np.matmul(M1,cog.T).T

    # Rotate so that TD1 and TD2 remain directly above ND1 and ND2.
    v=(cog[0]+cog[1])-(cog[2]+cog[3])
    theta=0.5*np.pi-np.arctan2(v[1],v[0])
    print( "    ...rotation angle determined from vector:", theta*180.0/np.pi ) 
    M2=np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    
    return np.matmul(M2, M1)
    
def standardise_coordinate_rotations(G, pos):
    # Identify the centers of geometry of the following SegIDS: TD1, TD2, ND1, ND2
    segIDList  = ['TD1', 'TD2', 'ND1', 'ND2']
    
    # First try to account for mirror imagery
    cogSegs=calculate_segID_cogs(G, pos, segIDList)
    TMat=calculate_transform_matrix(cogSegs)
  
    posOut = np.matmul(TMat,pos.T).T
        
    return posOut
        

In [ ]:
posNodesInit = compute_scaled_cylindrical_coordinates(dnad, offsetTheta=-0.5*np.pi)

In [ ]:
#posNodesInit[...,1].min()
for segid in ['LAS','RDO']:
    listIDs = [x for x in G.nodes() if G.nodes[x]['segid']==segid]
    print("Debug: domain %s theta distributions span between %g to %g" % (segid,
                                                                          np.min(posNodesInit[listIDs][...,0]),
                                                                          np.max(posNodesInit[listIDs][...,0])) )

In [ ]:
# Compute a single set of node position for all replicates, instead for one for each window
bPosSet=False
posNodes={}

In [ ]:
# Computed 2D embedding of atom positions based on the computed distance matrix in DNAD.
# Then rotate such that TMDs are on top and NBD1 is at the bottom-left hand corner
print("   ...Creating a 2D embedding based on the GCC matrices.")
if not bPosSet:
    bPosSet=True

    #for w in range(dnad.numWinds):
    #    distMat = dnad.distsAll[w]
    #    distMat[np.where(distMat<0)]=1.5*np.max(distMat)
    #    posEmbed = compute_graph_embedding(distMat, posNodesInit, bMetric=True)
    #    posEmbed = standardise_coordinate_rotations(dnad.nxGraphs[0], posEmbed)
    #    posNodes[w]={}
    #    for x in dnad.nxGraphs[w].nodes():
    #        posNodes[w][x]=posEmbed[x]
    #    print("    ...Done for GCC matrix in window %i" % w)
    
    distMat = np.zeros_like(dnad.distsAll[0])
    for w in range(dnad.numWinds):
        distMat += dnad.distsAll[w]
    distMat/=w
    distMat[np.where(distMat<0)]=1.5*np.max(distMat)

    posEmbed = compute_graph_embedding(distMat, posNodesInit)
    posEmbed = standardise_coordinate_rotations(dnad.nxGraphs[0], posEmbed)

    posNodesAverage={}
    for x in dnad.nxGraphs[0].nodes():
        posNodesAverage[x]=posEmbed[x]
    print("    ...Done for averaged GCC matrix")

In [ ]:
from bokeh.transform import transform
rangeX = rangeY = (0,dnad.numNodes)

#mat  = dnad.corrMatAll[w]
#data = np.array([ (i,j, mat[i,j]) for i in range(mat.shape[0]) for j in range(mat.shape[1]) if mat[i,j]>0])
#source = bokehModels.ColumnDataSource(data=dict(x=data[...,0], y=data[...,1], z=data[...,2]))
source = bokehModels.ColumnDataSource(data=dict(x=[],y=[],z=[], name=[]))
for u,v,z in dnad.nxGraphs[w].edges(data='weight'):
    source.data['x'].append(u) ; source.data['y'].append(v) ; source.data['z'].append(z)
    source.data['name'].append(dnad.nxGraphs[w].edges[u,v]['name'])    
    source.data['x'].append(v) ; source.data['y'].append(u) ; source.data['z'].append(z)
    source.data['name'].append(dnad.nxGraphs[w].edges[u,v]['name'])    

# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = bokehModels.LinearColorMapper(palette=colors, low=0, high=1)

p = bokehPlotting.figure(plot_width=800, plot_height=800, title="Generalised Correlation Coefficients matrix",
                        x_range=rangeX, y_range=rangeY)

p.rect(x="x", y="y", width=1, height=1, source=source,
       line_color=None, fill_color=transform('z', mapper))
                                      
tooltips = [("Name:", "@name")]
p.add_tools(bokehModels.HoverTool(tooltips=tooltips))

color_bar = bokehModels.ColorBar(color_mapper=mapper,
                     ticker=bokehModels.BasicTicker(desired_num_ticks=len(colors)),
                     formatter=bokehModels.PrintfTickFormatter(format="%.2f"))

p.add_layout(color_bar, 'right')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0

output_notebook()
show(p)

In [ ]:
import networkx.algorithms.community.quality as nxquality

# Creates a list of windows and order them according to graph modularity.
windModul = []
for w in range(dnad.numWinds):
    temp = OrderedDict()
    for k, v in dnad.nxGraphs[w].nodes(data='segid'):
        if v not in temp:
            temp[v]=set()
        temp[v].add(k)
    modul = nxquality.modularity(dnad.nxGraphs[w], [ temp[x] for x in temp ], weight="weight")
    windModul.append((w, modul))
    
windModul.sort(key=lambda x:x[1], reverse=True)

# Keep the window with the highest modularity as a reference for community matching
refWindow = windModul[0][0]

for w, mod in windModul:
    print( "Window {} has SegID modularity {:1.4f}.".format(w, mod) )

### Map the community IDs of each window to the one with the highest modularity.

In [ ]:
import networkx.algorithms.community.quality as nxquality

# Creates a list of windows and order them according to graph modularity.
windModul = []
for w in range(dnad.numWinds):
    modul = nxquality.modularity(dnad.nxGraphs[w], 
                         [ set(nodesList) for nodesList in dnad.nodesComm[w]["commNodes"].values()])
    windModul.append((w, modul))
    
windModul.sort(key=lambda x:x[1], reverse=True)

# Keep the window with the highest modularity as a reference for community matching
refWindow = windModul[0][0]

for w, mod in windModul:
    print( "Window {} has modularity {:1.4f}.".format(w, mod) )

In [ ]:
#print( dnad.nodesComm[w].keys() )
for w in range(dnad.numWinds):
    print( [ len(dnad.nodesComm[w]['commNodes'][x]) for x in dnad.nodesComm[w]['commOrderSize'] ] ) 

### Formatting: Define node and edge attributes for the renderer based on the data

Mostly definig a fized size and colour for each node property we want to show.

In [ ]:
def get_node_data_range(G, nodeAttr):
    vals = [ G.nodes[x][nodeAttr] for x in G.nodes() ]
    return np.min(vals), np.max(vals)

def get_node_color_label_map(G, nodeAttr):
    vals = [ G.nodes[x][nodeAttr] for x in G.nodes() ]
    _, i = np.unique(vals, return_index=True)
    vMap=np.array([ vals[x] for x in np.sort(i)])
    return vMap

def format_graph_nodes_by_palette(G, nodeAttr, palette, sizeRange, nullColour='#FFFFFF'):
    # Set the node properties as additional entries in the graph.
    # Should I wrap palette around for text encodings that has more types than the number of colours in palette
    pMax = len(palette)
    nodeSizes={} ; nodeColors={}    
    if type(G.nodes[0][nodeAttr])==str:
        vMap=get_node_color_label_map(G, nodeAttr)
        for x in G.nodes():
            i = np.where(vMap==G.nodes[x][nodeAttr])[0][0]
            if i>=pMax:
                nodeColors[x] = nullColour
            else:
                nodeColors[x] = palette[ i ]
            nodeSizes[x] = sizeRange
    else:
        vals = [ G.nodes[x][nodeAttr] for x in G.nodes() ]
        vMin = np.min(vals) ; vMax=np.max(vals)
        for x in G.nodes():
            r = (vals[x]-vMin)/(vMax-vMin)
            index = np.min( (int(r*pMax),pMax-1) )
            nodeColors[x] = palette[index]
            index = r*(sizeRange[1]-sizeRange[0])+sizeRange[0]
            nodeSizes[x] = index
    nx.set_node_attributes(G, nodeColors, "node_color_%s" % nodeAttr)
    #nx.set_node_attributes(G, nodeSizes, "node_size_%s" % nodeAttr)

In [ ]:
colourPaletteLin = bokehPalettes.Viridis256
#FF0000 #0072B2 #E69F00 #F0E442 #009E73 #56B4E9 #D55E00 #CC79A7 #000000 #666666 #FFFFFF
colourPaletteCat = ['#FF0000'] + list( bokehPalettes.Colorblind[8] ) + ['#666666']
maxCommunityColours=20
colourPaletteCommunity=bokehPalettes.Category20[maxCommunityColours]
loneCommunityColour='#FFFFFF'
sizeCircleDefault=10
sizeCircleRangeDefault=(8,16)
nodeAttrDefault='segid'
for w in range(dnad.numWinds):
    print("   ....running community ID assignment for window %i" % w)
    G=dnad.nxGraphs[w]
    format_graph_nodes_by_palette(G, 'segid', colourPaletteCat, sizeCircleDefault)
    format_graph_nodes_by_palette(G, 'bwcc', colourPaletteLin, sizeCircleRangeDefault)
    format_graph_nodes_by_palette(G, 'cwcc', colourPaletteLin, sizeCircleRangeDefault)    
    format_graph_nodes_by_palette(G, 'eigenvector', colourPaletteLin, sizeCircleRangeDefault)
    format_graph_nodes_by_palette(G, 'btws', colourPaletteLin, sizeCircleRangeDefault)
    # Note: These are the community IDs.
    #nColours=0
    #for x in dnad.nodesComm[w]['commLabels']:
    #    if len(dnad.nodesComm[w]['commNodes'][x]) > 1:
    #        nColours+=1
    k=0
    for i in dnad.nodesComm[w]['commOrderSize']:
        if len(dnad.nodesComm[w]['commNodes'][i]) > 1:
            if k>=maxCommunityColours:
                print(" = = WARNING: more than %i non-alone communitiess found! The palette is not enough." % maxCommunityColours)
                k=maxCommunityColours-1            
            for j in dnad.nodesComm[w]['commNodes'][i]:
                dnad.nxGraphs[w].nodes[j]['communityID']=k
                dnad.nxGraphs[w].nodes[j]['node_color_communityID']=colourPaletteCommunity[k]
                #dnad.nxGraphs[w].nodes[j]['node_size_communityID']=sizeCircleDefault
            k+=1

        else:
            j=dnad.nodesComm[w]['commNodes'][i][0]
            dnad.nxGraphs[w].nodes[j]['communityID']=-1
            dnad.nxGraphs[w].nodes[j]['node_color_communityID']=loneCommunityColour
            #dnad.nxGraphs[w].nodes[j]['node_size_communityID']=sizeCircleDefault
    
    # = = Set default node format
    for x in G.nodes():
        G.nodes[x]['node_color']=G.nodes[x]['node_color_%s' % nodeAttrDefault]
        #G.nodes[x]['node_size']=G.nodes[x]['node_size_%s' % nodeAttrDefault]
        G.nodes[x]['node_size']=10

In [ ]:
colourPaletteAnnotation = ('#FFFFFF', colourPaletteCat[3], colourPaletteCat[2], colourPaletteCat[0], colourPaletteCat[1])
colourMappingAnnotation = { 'lack':0, 'non':1, 'var':2, 'path':3, 'unk':4 }
print("   ....adding colours from annotation database...")
for x in dnad.nxGraphs[0].nodes:
    color = colourPaletteAnnotation[ colourMappingAnnotation[ dnad.nxGraphs[0].nodes[x]['annotation'] ] ]
    for w in range(dnad.numWinds):
        dnad.nxGraphs[w].nodes[x]['node_color_annotation'] = color
        #dnad.nxGraphs[w].nodes[x]['node_size_annotation'] = sizeCircleDefault

if allele != 'wt':
    for x in dnadWT.nxGraphs[0].nodes:
        color = colourPaletteAnnotation[ colourMappingAnnotation[ dnadWT.nxGraphs[0].nodes[x]['annotation'] ] ]
        for w in range(dnadWT.numWinds):
            dnadWT.nxGraphs[w].nodes[x]['node_color_annotation'] = color
            #dnad.nxGraphs[w].nodes[x]['node_size_annotation'] = sizeCircleDefault
    
if 'annotation' not in listNodeProperties:
    listNodeProperties.append('annotation')
    listNodePropertyNames.append('CFTR2 database annotation')

In [ ]:
print( G.nodes[0] )
print( G.edges(0) )
print( G.edges[0,1] )

### Defining custom interactive elements such as updating selections

In [ ]:
#def create_JS_selection_update(sourceFrom, sourcePos, sourceTo, tableTo):
def create_JS_broadcast_selection_updates(sourceFrom, sourcePos, sourceTargetNodes, sourceTargetNeighbours, sourceTargetEdges, tableTargetNodes, tableTargetNeighbours):
    """
    In-text example:
    graph_renderer.node_renderer.data_source.selected.js_on_change("indices", CustomJS(args=dict(,
    sFrom=graph_renderer.node_renderer.data_source  <- indices, names, segid
    pos=graph_renderer.layout_provider.graph_layout <- x,y
    sTo=sourceZoomNodes -> x,y, names, segid
    sAdj=sourceZoomAdjs -> x,y
    sEdge=sourceZoomEdges -> xs,ys
    table=table -> (none)
    """
    return bokehModels.CustomJS(args=dict(sFrom=sourceFrom,
                       #sEdgeFrom=sourceEdgeFrom,
                       pos=sourcePos,
                       sTo=sourceTargetNodes,
                       sAdj=sourceTargetNeighbours,
                       sEdge=sourceTargetEdges,
                       tTo=tableTargetNodes,
                       tAdj=tableTargetNeighbours),
        code="""
        var inds = cb_obj.indices;
        var dFrom = sFrom.data;
        //var dEdgeFrom = sEdgeFrom.data;
        var dTo = sTo.data;
        var dAdj = sAdj.data;
        var dEdges = sEdge.data;
        var listAdj = [] ; var indsAddedAdj = []  ;
        var xThis = 0 ; var yThis = 0 ; var xAdj = 0 ; var yAdj = 0 ; 
        dTo['x']     = [] ; dTo['y']      = [] ;
        dTo['name']  = [] ; dTo['segid']  = [] ; 
        dTo['node_color']  = []
        dAdj['x']    = [] ; dAdj['y']     = [] ;
        dAdj['name'] = [] ; dAdj['segid'] = [] ; 
        dAdj['node_color']  = []
        dEdges['xs'] = [] ; dEdges['ys']  = [] ;
        //dEdges['edge_alpha'] = [] ;
        for (var i = 0; i < inds.length; i++) {
        
            // Add position entries for node graph. Must keep all lists equal in length.
            xThis = pos[inds[i]][0] ; yThis = pos[inds[i]][1] ;    
            dTo['x'].push(xThis) ; dTo['y'].push(yThis) ;
            
            // Add metadata entries for node table. Must keep all lists equal in length.
            dTo['name'].push( dFrom['name'][inds[i]] ) ; dTo['segid'].push( dFrom['segid'][inds[i]] )
            dTo['node_color'].push( dFrom['node_color'][inds[i]] ) ;
            
            // Add metadatra entries for node table. Must keep all lists equal in length.            
            listAdj = dFrom['jsNeighbours'][inds[i]]
            for (var j = 0; j < listAdj.length; j++) {  
                if ( !inds.includes(listAdj[j]) ) {
                    xAdj = pos[listAdj[j]][0] ; yAdj = pos[listAdj[j]][1] ;
                    dEdges['xs'].push([xThis,xAdj]) ; dEdges['ys'].push([yThis,yAdj]) ;
                    //dEdges['edge_alpha'].push( )
                    if( !indsAddedAdj.includes(listAdj[j]) ) {
                        indsAddedAdj.push( listAdj[j] )
                        dAdj['x'].push(xAdj) ; dAdj['y'].push(yAdj)
                        dAdj['name'].push( dFrom['name'][listAdj[j]] ) ; dAdj['segid'].push( dFrom['segid'][listAdj[j]] ) ;
                        dAdj['node_color'].push( dFrom['node_color'][listAdj[j]] ) ;
                    }

                }
            }
        }
        
        //Tell all target sources and widgets to update 
        sTo.change.emit(); sAdj.change.emit();
        sEdge.change.emit();
        tTo.change.emit(); tAdj.change.emit();     
    """,
    )      

In [ ]:
def create_JS_update_format_nodes(source, listColourBar):
    """
    This Javascript snippet updates the node perperties and shows/hides the respective colourbar, depending on which property is being called for.
    """
    return bokehModels.CustomJS(args=dict(s1=source, lcb=listColourBar),
        code="""
        var d1 = s1.data;
        var f  = this.item;
        for (var i = 0; i < s1.length; i++) {
            d1['node_color'][i] = d1['node_color_'+f][i]
            // d1['node_size'][i] = d1['node_size_'+f][i]
        }
        s1.change.emit();
        for (var i = 0; i < lcb.length; i++) {
            if ( lcb[i].name == f ) {
                lcb[i].visible = true
            } else {
                lcb[i].visible = false
            }
        }
    """,
    )

def create_JS_update_format_edges(source):
    """
    This Javascript snippet updates the edge perperties, depending on which property is being called for.
    """
    return bokehModels.CustomJS(args=dict(s1=source),
        code="""
        var d1 = s1.data;
        var f  = this.item;
        //console.log(f, 'edge_alpha_'+f)
        for (var i = 0; i < s1.length; i++) {
            d1['edge_alpha'][i] = d1['edge_alpha_'+f][i]
        }
        s1.change.emit();
    """,
    )

    
def create_JS_search_attributes_and_select(source, attr='name', charSplit=',', listCallbacks = []):
        return bokehModels.CustomJS(args=dict(s=source, attr=attr, charSplit=charSplit, listCB = listCallbacks), code="""
        var input  = cb_obj.value         ;
        var d      = s.data               ;
        var prevID = s.selected.indices   ;
        
        var list = input.split(charSplit)
        s.selected.indices = [] ;
        for (var i = 0 ; i < list.length ; i++) {
            var j = d[attr].indexOf(list[i])
            if ( j > -1 ) {
                s.selected.indices.push(j) ;
            }
        }
        s.change.emit ;
        // Trigger downstream callbacks
        for (var i = 0 ; i < listCB.length ; i++) {
            listCB[i].execute(cb_obj = s.selected ) ;
        }        
        """
        )
    
def create_JS_clone_selections(listSources, listCallbacks):
    return bokehModels.CustomJS(args=dict(listS = listSources, listCB = listCallbacks), code="""
    for (var i = 1 ; i < listCB.length ; i++) {
            listS[i].selected.indices = listS[0].selected.indices ;
            listCB[i].execute(cb_obj = listS[i].selected ) ;
    }        
    """
    )

## Finally prepare the plot

In [ ]:
# Define plot with starting X and Y ranges, kind of tools available, etc.
lineWidthDefault=1
hoverColour='blue' ; lineWidthHover=4

pWidth=800
pHeight=700
sbHeight=50
tWidth=150 ; tHeight=int(0.5*(pHeight-sbHeight))
wZoomFig=300                    ; hZoomFig=wZoomFig
wZoomTableTitle=int(wZoomFig/2) ; hZoomTableTitle=50
wZoomTab=int(wZoomFig/2)        ; hZoomTab=pHeight-hZoomFig-hZoomTableTitle
sWidth=int(pWidth/2) ; sHeight=int(pHeight/2)
ssWidth=int(pWidth/3)

# = = = = = = = = = = = = = = = = = = = = = = = = = = =
# = = Widgets to link node and edge properties across all DNA windows: Initial declaration.
# = = = = = = = = = = = = = = = = = = = = = = = = = = =

# = = SIZE, EDGE, AND POSITION PROPERTIES = = = 
#spinnerWidget1 = bokehModels.Slider(title='Node size', start=0, end=20, step=0.5, value=sizeCircleDefault)
spinnerWidget1 = bokehModels.Spinner(title='Node size', low=0, high=20, step=0.5, value=sizeCircleDefault)
spinnerWidget2 = bokehModels.Spinner(title='Edge width', low=0, high=10, step=0.5, value=lineWidthDefault)
spinnerWidget3 = bokehModels.Spinner(title='Selection line width', low=0, high=10, step=0.5, value=lineWidthHover)
spinnerWidget4 = bokehModels.Spinner(title='Window-based node positions', low=0, high=1, step=0.05, value=0.0)
widgetSliders = bokehLayouts.row([spinnerWidget1,spinnerWidget2,spinnerWidget3, spinnerWidget4])

# = = = = = = = = = = = = = = = = = = = = = = = = = = =
# = = NODE-BASED COLOUR CONTROLS = = = 
# = = = = = = = = = = = = = = = = = = = = = = = = = = =
#dictButtons={}
#for nodeAttr in listNodeProperties:
#    b = bokehModels.Button(width_policy='min', label="format by %s" % nodeAttr)
#    dictButtons[nodeAttr] = b
#widgetColourControls=bokehLayouts.row([ dictButtons[x] for x in dictButtons.keys() ])
menuNodeColourControls = []
for name, attr in zip(listNodePropertyNames, listNodeProperties):
    menuNodeColourControls.append( (name, attr) )
dropdownNodeColourControls = bokehModels.Dropdown(label="Colour nodes by...", button_type="default", width_policy='min',
                                                  background="#ddddee", menu=menuNodeColourControls)

menuEdgeColourControls = [('Edge-betweenness','btws'),('Gen. correlation coefficient','weight')]
dropdownEdgeColourControls = bokehModels.Dropdown(label="Shade edges by...", button_type="default", width_policy='min',
                                                  background="#ddddee", menu=menuEdgeColourControls)

buttonCloneSelection = bokehModels.Button(width_policy='min', label="Apply Window 0 selection to all windows.")

widgetColourControls = bokehLayouts.row( [dropdownNodeColourControls,dropdownEdgeColourControls,buttonCloneSelection] )



# = = = TABS Setup for each window
bFirst=True
xRangeInit=yRangeInit=(-3,3)
dictGraphRenderers={}
dictTab={} ; dictFigures={} ; dictZoomedFigures={} ; dictTableNodes={} ; dictTableEdges={}
listGraphNodeSources = [] ; listCallbackBigs = [] ; 
for w in range(dnad.numWinds):
    G = dnad.nxGraphs[w]
    # = = = Set up linked Panning by giving the same range to all figures.
    if bFirst:
        dictFigures[w] = bokehPlotting.figure(title="%s %s simulations, window %i" % (allele, temperature, w),
                plot_width=pWidth, plot_height=pHeight,
                x_range=xRangeInit, y_range=yRangeInit,
                tools = "pan,wheel_zoom,box_zoom,box_select,tap,save,reset,help",
                toolbar_location='above')
        pFirst=dictFigures[w]
    else:
        dictFigures[w] = bokehPlotting.figure(title="%s %s simulations, window %i" % (allele, temperature, w),
                plot_width=pWidth, plot_height=pHeight,
                x_range=pFirst.x_range, y_range=pFirst.y_range,
                tools = "pan,wheel_zoom,box_zoom,box_select,tap,save,reset,help",
                toolbar_location='above')

    # Auto or pre-calc coordinates.
    
    #graphRenderer = bokehPlotting.from_networkx(G, posNodes[w], scale=2, center=(0,0))
    graphRenderer = bokehPlotting.from_networkx(G, posNodesAverage, scale=2, center=(0,0))
    #graphRenderer = bokehPlotting.from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
    dictGraphRenderers[w] = graphRenderer
    
    # Set up the tooltip on mouse hover here
    hoverTool1 = bokehModels.HoverTool(tooltips=[("name", "@name")], point_policy='snap_to_data')
    #hoverTool1 = bokehModels.HoverTool(tooltips=[("name", "@name"), ("%s" % nodeAttr,"@%s" % nodeAttr)], point_policy='snap_to_data')
    #hoverTool2 = bokehModels.HoverTool(renderers=[graphRenderer.edge_renderer],
    #                                  tooltips=[("name", "@name"), ("btws", "@btws")], point_policy='snap_to_data')
    #hoverTool = bokehModels.HoverTool(tooltips=[("name", "@name")], point_policy='snap_to_data')
    dictFigures[w].add_tools(hoverTool1)
    #dictFigures[w].add_tools(hoverTool2)
    # Turn Wheel Zoom on by default
    dictFigures[w].toolbar.active_scroll = dictFigures[w].select_one(bokehModels.WheelZoomTool)
       
    # = = FORMATTING COLOURBARS.
    listColourBars=[]
    for nodeAttr in listNodeProperties:
        if nodeAttr == 'segid':
            colourStyle = "node_color_%s" % nodeAttr
            #colourList = get_node_color_label_map(G, nodeAttr)[:len(colourPaletteCat)]
            colourList = get_node_color_label_map(G, nodeAttr)
            palette = colourPaletteCat
            if len(colourList) > len(palette):
                colourList = colourList[:len(palette)]
            elif len(palette) > len(colourList):
                palette = colourPaletteCat[:len(colourList)]
            colourMapper = bokehModels.mappers.CategoricalColorMapper(palette=palette, factors=colourList)
            colourBar = bokehModels.ColorBar(name=nodeAttr, color_mapper=colourMapper, label_standoff=12)
            dictFigures[w].add_layout(colourBar, 'right')        
        elif nodeAttr == 'communityID':
            colourStyle = "node_color_%s" % nodeAttr
            colourList = [str(x) for x in range(len(colourPaletteCommunity))]
            colourMapper = bokehModels.mappers.CategoricalColorMapper(palette=colourPaletteCommunity, factors=colourList)
            colourBar = bokehModels.ColorBar(name=nodeAttr, color_mapper=colourMapper, label_standoff=12, title=nodeAttr)
            dictFigures[w].add_layout(colourBar, 'right')
        elif nodeAttr == 'annotation':
            colourStyle = "node_color_%s" % nodeAttr
            colourList = [k for k in colourMappingAnnotation.keys()]
            colourMapper = bokehModels.mappers.CategoricalColorMapper(palette=colourPaletteAnnotation, factors=colourList)
            colourBar = bokehModels.ColorBar(name=nodeAttr, color_mapper=colourMapper, label_standoff=12, title=nodeAttr)
            dictFigures[w].add_layout(colourBar, 'right')
        else:
            cMin, cMax = get_node_data_range(G, nodeAttr)
            colourStyle = bokehTransform.linear_cmap(nodeAttr, colourPaletteLin,cMin,cMax)
            colourMapper = bokehModels.mappers.LinearColorMapper(palette=colourPaletteLin, low=cMin, high=cMax)
            colourBar = bokehModels.ColorBar(name=nodeAttr, color_mapper=colourMapper, label_standoff=12, title=nodeAttr)
            dictFigures[w].add_layout(colourBar, 'right')
        colourBar.visible=False
        listColourBars.append(colourBar)
    for cb in listColourBars:
        if cb.name == nodeAttrDefault:
            cb.visible = True
    
    # = = = RENDERER Setup.
    #graphRenderer.node_renderer.glyph = bokehModels.Circle(size="node_size", fill_color=cMapTest, line_color='black', line_width=lineWidthDefault)
    graphRenderer.node_renderer.glyph = bokehModels.Circle(size="node_size", fill_color="node_color", line_width=lineWidthDefault)
    graphRenderer.node_renderer.nonselection_glyph = bokehModels.Circle(fill_alpha=0.0, line_alpha=0.0)
    graphRenderer.node_renderer.selection_glyph = bokehModels.Circle(size="node_size", fill_color="node_color", line_width=lineWidthHover)
    graphRenderer.node_renderer.hover_glyph = bokehModels.Circle(fill_color="node_color", line_width=lineWidthHover)
    
    #if nodeAttr == 'segid':
    #    vMap = get_node_color_label_map(G, nodeAttr )
    #    legend = bokehModels.Legend(items=[ (x, [graphRenderer.node_renderer.glyph]) for x in vMap ],
    #        location='top_left', orientation='horizontal',
    #        border_line_color="black", title='')
    #    dictFigures[nodeAttr].add_layout(legend, 'center')
    
    # help(dictFigures[nodeAttr].renderers[0].node_renderer.data_source)
    
    graphRenderer.edge_renderer.glyph = bokehModels.MultiLine(line_color="black", line_alpha="edge_alpha", line_width=lineWidthDefault)
    #graphRenderer.edge_renderer.nonselection_glyph = bokehModels.MultiLine(line_alpha=0.0)
    graphRenderer.edge_renderer.selection_glyph = bokehModels.MultiLine(line_color="black", line_alpha="edge_alpha", line_width=lineWidthHover)
    graphRenderer.edge_renderer.hover_glyph = bokehModels.MultiLine(line_color=hoverColour, line_alpha="edge_alpha", line_width=lineWidthHover)

    graphRenderer.selection_policy = bokehModels.NodesAndLinkedEdges()
    #graphRenderer.selection_policy = bokehModels.EdgesAndLinkedNodes()
    graphRenderer.inspection_policy = bokehModels.NodesAndLinkedEdges()
    #graphRenderer.inspection_policy = bokehModels.EdgesAndLinkedNodes()
    #graphRenderer.inspection_policy = bokehModels.NodesOnly()

    # = = NODE-BASED COLOUR CONTROLS = = = 
    #for nodeAttr in listNodeProperties:
    #    dictButtons[nodeAttr].js_on_event(bokehEvents.ButtonClick,
    #                          create_JS_update_format_bynode(graphRenderer.node_renderer.data_source, nodeAttr, listColourBars))
    dropdownNodeColourControls.js_on_event("menu_item_click",
                                           create_JS_update_format_nodes(graphRenderer.node_renderer.data_source, listColourBars))
    dropdownEdgeColourControls.js_on_event("menu_item_click",
                                           create_JS_update_format_edges(graphRenderer.edge_renderer.data_source))
    dictFigures[w].renderers.clear()
    dictFigures[w].renderers.append(graphRenderer)

    # = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # = = Widget Javascripts to link node and edge properties across all DNA windows
    # = = = = = = = = = = = = = = = = = = = = = = = = = = =
    
    #spinnerWidget1 = bokehModels.Slider(title='Node size', start=0, end=20, step=0.5, value=12, width=ssWidth, width_policy='fit')
    spinnerWidget1.js_link('value_throttled', graphRenderer.node_renderer.glyph, 'size')
    #spinnerWidget2 = bokehModels.Slider(title='Edge width', start=0, end=10, step=0.5, value=lineWidthDefault, width=ssWidth, width_policy='fit')
    spinnerWidget2.js_link('value_throttled', graphRenderer.edge_renderer.glyph, 'line_width')
    #spinnerWidget3 = bokehModels.Slider(title='Selection line width', start=0, end=10, step=0.5, value=lineWidthHover, width=ssWidth, width_policy='fit')
    spinnerWidget3.js_link('value_throttled', graphRenderer.node_renderer.selection_glyph, 'line_width')
    spinnerWidget3.js_link('value_throttled', graphRenderer.edge_renderer.selection_glyph, 'line_width')   

    # = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # = = Table column for selecting individual nodes
    # = = = = = = = = = = = = = = = = = = = = = = = = = = =
    
    dataColumns1 = [ bokehModels.TableColumn(field="name", title="Name", width=50) ]
    dataColumns1.append( bokehModels.TableColumn(field='segid', title="%s" % "SegID", width=100))
    #for nodeAttr in listNodeProperties:
    #    dataColumns1.append( bokehModels.TableColumn(field=nodeAttr, title="%s" % nodeAttr) )
    tableWidget1 = bokehModels.DataTable(source=graphRenderer.node_renderer.data_source, columns=dataColumns1,
            width=tWidth, height=tHeight, sortable=True, selectable=True, editable=False )
    
    dataColumns2 = [ bokehModels.TableColumn(field="name", title="Name", width=50), bokehModels.TableColumn(field='btws', title='Edge betweenness centrality', width=100)]
    tableWidget2 = bokehModels.DataTable(source=graphRenderer.edge_renderer.data_source, columns=dataColumns2,
            width=tWidth, height=tHeight, sortable=True, selectable=True, editable=False )
    dictTableNodes[w]=tableWidget1
    dictTableEdges[w]=tableWidget2
    # height = dictFigures[nodeAttr].plot_height, height_policy='fixed'

    # = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # = = Zoom-in plots for a detailed view of selected nodes and their neighbours
    # = = = = = = = = = = = = = = = = = = = = = = = = = = =

    sourceZoomNodes = bokehModels.ColumnDataSource(data=dict(x=[], y=[], name=[], segid=[], node_color=[]))
    sourceZoomAdjs  = bokehModels.ColumnDataSource(data=dict(x=[], y=[], name=[], segid=[], node_color=[]))
    sourceZoomEdges = bokehModels.ColumnDataSource(data=dict(xs=[], ys=[]))
    
    figZoom = bokehPlotting.figure(title="Selection view", plot_width=wZoomFig, plot_height=hZoomFig,
                                      tools=["box_zoom", "wheel_zoom", "reset", "save"])
   
    glyphEdges = bokehModels.MultiLine(xs="xs", ys="ys", line_color="#000000", line_alpha=0.6, line_width=2)
    figZoom.add_glyph(sourceZoomEdges, glyphEdges)
    glyphAdj = figZoom.circle("x", "y", source=sourceZoomAdjs, size=10, alpha=0.6, color='node_color',
                           line_width=1, line_color='black')
    #glyphSel = figZoom.circle("x", "y", source=sourceZoomNodes, size=15, alpha=1, color='node_color',
    #                       line_width=1.5, line_color='black')
    glyphSel = figZoom.star(x="x", y="y", source=sourceZoomNodes, size=20, alpha=1, color='node_color',
                            line_width=1.5, line_color='black')    
    dictZoomedFigures[w]=figZoom

    tooltips = [("Name:", "@name"), ("SegID:", "@segid")]
    figZoom.add_tools(bokehModels.HoverTool(tooltips=tooltips, renderers=[glyphSel,glyphAdj]))    
    figZoom.toolbar.active_scroll = figZoom.select_one(bokehModels.WheelZoomTool)
    
    # The div is here to provde a short title.
    divZoomNodes = bokehModels.Div( width=wZoomTableTitle, height=hZoomTableTitle, text="<strong>Selected Nodes</strong>" )
    columnsZoomNodes = [ bokehModels.TableColumn(field="name", title="Name", width=50), bokehModels.TableColumn(field="segid", title="SegID", width=100)]
    tableZoomNodes = bokehModels.DataTable(source=sourceZoomNodes, columns=columnsZoomNodes,
            width=wZoomTab, height=hZoomTab, sortable=True, selectable=False, editable=False )
    
    divZoomNeighbours = bokehModels.Div( width=wZoomTableTitle, height=hZoomTableTitle, text="<strong>Node Neighbours</strong>" )
    columnsZoomNeighbours = [ bokehModels.TableColumn(field="name", title="Name", width=50), bokehModels.TableColumn(field="segid", title="SegID", width=100)]
    tableZoomNeighbours = bokehModels.DataTable(source=sourceZoomAdjs, columns=columnsZoomNeighbours,
            width=wZoomTab, height=hZoomTab, sortable=True, selectable=False, editable=False )     
   
    widgetZoomPackage = bokehLayouts.column(figZoom,
                                            bokehLayouts.row(bokehLayouts.column(divZoomNodes,tableZoomNodes),
                                                             bokehLayouts.column(divZoomNeighbours,tableZoomNeighbours),
                                                            )
                                           )
    
    # Link selection of nodes in main plot to these zoom in windows.
    callbackBig = create_JS_broadcast_selection_updates(sourceFrom=graphRenderer.node_renderer.data_source,
        sourcePos=graphRenderer.layout_provider.graph_layout,
        sourceTargetNodes=sourceZoomNodes,
        sourceTargetNeighbours=sourceZoomAdjs,
        sourceTargetEdges=sourceZoomEdges,
        tableTargetNodes=tableZoomNodes,
        tableTargetNeighbours=tableZoomNeighbours,
        )
    graphRenderer.node_renderer.data_source.selected.js_on_change("indices", callbackBig)

    # = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # = = Search bar for selecting specific residues
    # = = = = = = = = = = = = = = = = = = = = = = = = = = =
    #textBanner = Paragraph(text=welcome_message, width=200, height=100)
    inputSearchbox = bokehModels.TextInput(value="", title="Find (Name, Name,...):", width=tWidth, height=sbHeight )
    inputSearchbox.js_on_change('value', create_JS_search_attributes_and_select(source=graphRenderer.node_renderer.data_source,
                                                                                attr='name',
                                                                                listCallbacks = [ callbackBig ] ) )
    
    # = = = Selection synchronizing WIP
    listGraphNodeSources.append(graphRenderer.node_renderer.data_source)
    listCallbackBigs.append(callbackBig)
    
    #graphRenderer.node_renderer.data_source.selected.js_on_change("indices",
    #            create_JS_selection_update(graphRenderer.node_renderer.data_source, posNodes, sourceSubplotNodes, tableWidget3) )
    
    # = = FINAL TAB CONTENTS: Assemble all widgets into a single tab.
    tabContents=bokehLayouts.row(dictFigures[w],
                                 bokehLayouts.column(inputSearchbox,tableWidget1,tableWidget2),
                                 widgetZoomPackage,
                                )
    #tabContents=bokehLayouts.row( dictFigures[w],
    #                              bokehLayouts.column(tableWidget1,tableWidget2),
    #                              )
    
    dictTab[w] = bokehModels.Panel(child=tabContents, title="Window %s" % w)
    
    if bFirst:
        bFirst=False

In [ ]:
#print( graphRenderer.__dict__.keys() )
#graphRenderer.node_renderer.data_source.data.keys()
#graphRenderer.edge_renderer.data_source.data.keys()
#colourBar.name
#tableWidget1.source.data.keys()
#graphRenderer.layout_provider.properties()

In [ ]:
buttonCloneSelection.js_on_click( create_JS_clone_selections(listGraphNodeSources, listCallbackBigs) )

In [ ]:
# = = Linking Node positions = = Note: currently doesn't work even though the positions are updated.
dictGraphPos={} ; dictGraphLayout={}
for w in range(dnad.numWinds):
    dictGraphLayout[w]=graphRenderer.layout_provider
    dictGraphPos[w]=graphRenderer.layout_provider.graph_layout

spinnerWidget4.js_property_callbacks={}
spinnerWidget4.js_on_change('value_throttled',
    bokehModels.CustomJS(args=dict(pMean=posNodesAverage,
                                   pWindow=posNodes,
                                   pTarget=dictGraphPos,
                                   gTarget=dictGraphLayout,
                                   ),
    code="""
        var v = cb_obj.value_throttled ;
        console.log('init:', pTarget[0][0])
        for (var w in pTarget ) {
            for (var i in pTarget[w] ) {
                pTarget[w][i][0] = pMean[i][0]+v*(pWindow[w][i][0]-pMean[i][0])
                pTarget[w][i][1] = pMean[i][1]+v*(pWindow[w][i][1]-pMean[i][1])
            }
            gTarget[w].change.emit;
        }
        console.log('final:', pTarget[0][0])
    """,
    ))

In [ ]:
#selectWidget = bokehModels.Select(title="Select Colour Map", value="node_color_bwcc",
#                options = ["node_color_bwcc","node_color_eigenvector", "node_color_segid"])
#selectWidget.js_link('value', graphRenderer.node_renderer.glyph, 'fill_color')
#graphRenderer.node_renderer.data_source.column_names

In [ ]:
#controlWidget = bokehLayouts.WidgetBox(selectWidget)

## Create a separate tab dedicated to graphical analysis

### Sequence-based graphs

In [ ]:
def extract_node_data(dnad, nodeAttr):
    out=np.zeros( (dnad.numWinds,dnad.numNodes) )
    for w in range(dnad.numWinds):
        for x in dnad.nxGraphs[w].nodes():
            out[w,x] = dnad.nxGraphs[w].nodes[x][nodeAttr]
    return out

In [ ]:
colourPaletteLines = bokehPalettes.Colorblind[8]
#listNodeProperties=['segid','communityID','bwcc','eigenvector', 'btws']

def generate_plot_data_source(dnad):
    s = bokehModels.ColumnDataSource(data=dict())
    listNodeIDs = [ x for x in dnad.nxGraphs[0].nodes() ]
    s.data['x']     = listNodeIDs
    s.data['name']  = [ dnad.nxGraphs[0].nodes[x]['name'] for x in listNodeIDs ]
    s.data['segid'] = [ dnad.nxGraphs[0].nodes[x]['segid'] for x in listNodeIDs ]
    return s

TOOLTIPS = [("name", "@name"),("segid", "@segid")]

bFirst=True
listGraphs=[] ; dictHT={}
for nodeAttr, nodeAttrName in zip(listNodePropertiesPlot,listNodePropertiesPlotNames):
    if bFirst:
        plotAttr = bokehPlotting.Figure(plot_width=1000, plot_height=250, x_axis_label='Node ID', y_axis_label=nodeAttr,
                                   title=nodeAttrName, tooltips=TOOLTIPS, tools = "pan,xwheel_zoom,box_zoom,save,reset,help")
        pFirst=plotAttr
    else:
        plotAttr = bokehPlotting.Figure(plot_width=1000, plot_height=250, x_axis_label='Node ID', y_axis_label=nodeAttr, x_range=pFirst.x_range,
                                   title=nodeAttrName, tooltips=TOOLTIPS, tools = "pan,xwheel_zoom,box_zoom,save,reset,help")
    plotAttr.toolbar.active_scroll = plotAttr.select_one(bokehModels.WheelZoomTool)
    dataBlock = extract_node_data(dnad, nodeAttr)    
    plotSource = generate_plot_data_source(dnad)
    # = = Plot per window
    for w in range(dnad.numWinds):
        yStr='y%i' % w
        plotSource.data['y%i'%w]=dataBlock[w]
        renderer = plotAttr.line('x', yStr, legend_label='window %i' % w, source=plotSource,line_width=1, line_color=colourPaletteLines[w])
        renderer.visible = True
    # = = Plot mean values
    plotSource.data['mean']=np.mean(dataBlock,axis=0)
    renderer = plotAttr.line('x', 'mean', legend_label='mean', source=plotSource,line_width=1, line_color='black')
    renderer.visible = False
    # Plot mean wildtype values
    if allele != "wt":
        plotSourceWT = generate_plot_data_source(dnadWT)
        dataBlockWT = extract_node_data(dnadWT, nodeAttr)
        plotSourceWT.data['mean_WT']=np.mean(dataBlockWT,axis=0)
        renderer = plotAttr.line('x', 'mean_WT', legend_label='mean (WT)', source=plotSourceWT,line_width=1, line_color='grey')
        renderer.visible = False
    plotAttr.legend.location = "top_left"
    plotAttr.legend.click_policy="hide"    
    listGraphs.append(plotAttr)
    bFirst=False

tabContents = bokehLayouts.column(listGraphs)
    
dictTab['graph'] = bokehModels.Panel(child=tabContents, title="Graphs")

### Annotation-based graphs

In [ ]:
from scipy.stats.kde import gaussian_kde

def ridge(category, data, scale=20):
    return list(zip([category]*len(data), scale*data))

xCats = list(colourMappingAnnotation.keys())  
dictAnnotationTab={}

bFirst=True
for nodeAttr in ['eigenvector','bwcc','btws']:    
    fig = bokehPlotting.Figure(plot_width=1000, plot_height=250, y_axis_label=nodeAttr,
                               tools="", background_fill_color="#efefef", x_range=xCats, toolbar_location=None)
    listYVals=[]

    for i,k in enumerate(xCats):
        yVals = []
        for w in [0]:
            G = dnadWT.nxGraphs[w]
            yy = [ G.nodes[x][nodeAttr] for x in G.nodes() if G.nodes[x]['annotation'] == k ]
            yVals.extend( yy )
        listYVals.append(yVals)
           
    yMax = np.max( [np.max(x) for x in listYVals] ) 
    nPoints=200 ; xScale=0.005                      
    yGrids = np.linspace(0.0,yMax, nPoints)
    yGridsPlot = np.concatenate( (yGrids,np.flip(yGrids)) )
    source = bokehModels.ColumnDataSource(data=dict(y=yGridsPlot))              
    
    for i,k in enumerate(xCats):
        pdf = gaussian_kde(listYVals[i])
        xCurve = pdf(yGrids)
        #xCurvePlot = np.insert(xCurve,(0,nPoints),(0, 0))
        xCurvePlot = np.concatenate( (xCurve,-1*np.flip(xCurve)) )
        xVals = ridge(k, xCurvePlot, scale=xScale)
        source.add(xVals, k)
        fig.patch(k, 'y', color=colourPaletteAnnotation[i], alpha=0.6, line_color="black", source=source)

    fig.outline_line_color = None
    fig.background_fill_color = "#efefef"
    #fig.yaxis.ticker = bokehModels.FixedTicker(ticks=list(range(0, 1, 10)))
    #fig.yaxis.formatter = bokehModels.PrintfTickFormatter(format="%d%%")
    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = "#dddddd"
    fig.ygrid.ticker = fig.yaxis.ticker
    fig.axis.minor_tick_line_color = None
    fig.axis.major_tick_line_color = None
    fig.axis.axis_line_color = None
    fig.x_range.range_padding = 0.12

    # = = Additional Annotations
    if bFirst:
        counts=[ 'N = %i' % len(x) for x in listYVals ]
        sourceAnnotationLabel=bokehModels.ColumnDataSource(data=dict(x=xCats, label=counts) )
        
    labels = bokehModels.LabelSet(x='x', y=yMax, text='label',
                                  x_offset=10, y_offset=-10, source=sourceAnnotationLabel)
    fig.add_layout(labels)
    dictAnnotationTab[nodeAttr] = fig
    
    bFirst=False

dictTab['CFTR2'] = bokehModels.Panel(title="CFTR2-wildtype correlations",
                                     child=bokehLayouts.column( [dictAnnotationTab[x] for x in dictAnnotationTab.keys()]))
#output_notebook()
#show(bokehLayouts.column( [dictAnnotationTab[x] for x in dictAnnotationTab.keys()]) )

## Show or export the final analysis interface

In [ ]:
if bExportHTML:
    output_file(outputFileName)
else:
    output_notebook()

In [ ]:
#show(bokehModels.Tabs(tabs=[dictTab[x] for x in dictTab.keys()]))
show(bokehLayouts.column(widgetSliders,widgetColourControls,bokehModels.Tabs(tabs=[dictTab[x] for x in dictTab.keys()])))